In [2]:
%load_ext autoreload
%autoreload 2
from expressiveness_benchmark.types import Plan, Task, Language, SourceRange, Program
from code_widget.example import CodeWidget
from dataclasses import replace
import json
import pandas as pd

In [3]:
# CHANGE ME!
TASK_ID = 'exists_unique'
AUTHOR = 'g'


In [4]:
task = Task(
    id=TASK_ID,
    description="Find if there exists an object that is different than all other objects",
    plan=[
    ],
    sample_input={
       "list":[{"id":0,"value":1},{"id":1,"value":2},{"id":2,"value":3},{"id":3,"value":1}]
    },
    sample_output=1,
    category='Basic'
)
task.save()

prototype = Program(
    task=TASK_ID,
    author=AUTHOR,
    language=''    
)

In [ ]:
imp = replace(prototype,
    language='python-imperative',
    source='''
def exists_unique(list):
    for elem in list:
        e = elem["value"]
        if list.count(elem) == 1:
            return 1
    return 0
''')
imp.execute(task)
imp.save()

In [ ]:
func = replace(prototype,
    language='python-functional',
    source='''
def exists_unique(list):
    return int(any(list.count(x) == 1 for x in list))
''')
func.execute(task)
func.save()

In [ ]:
func = replace(prototype,
    language='python-pandas',
    source='''
def exists_unique(list):
    values = list.value.value_counts()
    if 1 in values:
        return 1
    return 0
''')
func.execute(task)
func.save()

In [ ]:
func = replace(prototype,
    language='sql',
    source='''
    SELECT Count(*) > 0 AS HasUnique
    FROM (SELECT DISTINCT value FROM list)
''')
func.execute(task)
func.save()

In [7]:
datalog = replace(prototype,
    language='datalog',
    source='''
.decl is_unique(Value:number)
is_unique(Value) :-
list(_,Value), !not_unique(Value).
.decl not_unique(Value:number)
not_unique(Value) :- list(Id1,Value), list(Id2,Value), Id1 != Id2.

exists_unique(Value):-
(is_unique(_),Value=1);
(!is_unique(_),Value=0).

''').load_plan()
datalog.execute(task)
datalog.save()